# Purpose of Notebook

To conduct a PoC with dummy data to carry out the below highlevel process:

1. Read Data from a .xslx and import into pandas data frame
2. Apply the required business rules to categorise information in python
3. Provide the output in a .xslx file that is split accordingly

## Step 1 - Read Data from .xslx and import into pandas data frame

Install the required libraries for the script

In [1]:
#!pip install pandas
#!pip install ipywidgets
#!pip install openpyxl


Read the data from .xslx and import into pandas

In [2]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl import load_workbook

# Read the data from the Excel file, skipping the first row

df = pd.read_excel('./data.xlsx', sheet_name='2024 Tracking', skiprows=1)



In [3]:
### This can be removed once validated
df["Average Monthly Reach Projected"].head(10)
### This can be removed once validated

0    0.500
1    0.600
2    0.680
3    0.700
4    0.745
5    0.750
6    0.800
7    0.350
8    0.400
9      NaN
Name: Average Monthly Reach Projected, dtype: float64

## Step 2 - We need to add the required business logic

In [4]:
# Need to add required business logic to the data. Using the column "Average Monthly Reach Projected"
# I will categorise the reach into 3 categories:
# 1. Under Invested - If the reach is less than 70%
# 2. Correctly Invested - If the reach is between 70% and 75%
# 3. Over Invested - If the reach is more than 75%

def categorize_reach(x):
    if pd.isnull(x):
        return x
    elif x > 0.75:
        return '3 - Over Invested'
    elif x >= 0.70:
        return '2 - Correctly Invested'
    else:
        return '1 - Under Invested'

df['Monthly Reach Grouping'] = df['Average Monthly Reach Projected'].apply(categorize_reach)

# Step 3 - Now we prepare the data to be loaded into the .xslx output

For this we will need 4 dataframe outputs
1. All rows and data
2. Only those where they are 1 - Under Invested
3. Only those where they are 2 - Correctly Invexted
4. Only those where they are 3 - Over Invested
5. Those where % is blank or nan

In [5]:
# Create a new DataFrame for the columns required in the output
df_filtered_columns = df[['Market', 'Brand', 'Category', 'Average Monthly Reach Target', 'Average Monthly Reach Projected', 'Monthly Reach Grouping']]
df_filtered_columns.head(10)

,Market,Brand,Category,Average Monthly Reach Target,Average Monthly Reach Projected,Monthly Reach Grouping
0,United States,Adult Motrin,Pain,0.7,0.500,1 - Under Invested
1,United States,Adult Tylenol,Pain,0.7,0.600,1 - Under Invested
2,United States,Adult Zyrtec,Allergy Care,0.7,0.680,1 - Under Invested
3,United States,Aveeno,Body,0.7,0.700,2 - Correctly Invested
4,United States,Aveeno,Face,0.7,0.745,2 - Correctly Invested
5,United States,Aveeno,Megabrand,0.7,0.750,2 - Correctly Invested
6,United States,Aveeno Baby,Baby,0.7,0.800,3 - Over Invested
7,United States,Aveeno Kids,Baby,0.7,0.350,1 - Under Invested
8,United States,Band Aid,Would Care,0.7,0.400,1 - Under Invested
9,United States,Benadryl,Allergy Care,0.7,NaN,NaN


In [6]:
# Create a data frame for each required output
df_under_invested = df_filtered_columns[df_filtered_columns['Monthly Reach Grouping'] == '1 - Under Invested']
df_correctly_invested = df_filtered_columns[df_filtered_columns['Monthly Reach Grouping'] == '2 - Correctly Invested']
df_over_invested = df_filtered_columns[df_filtered_columns['Monthly Reach Grouping'] == '3 - Over Invested']
df_nan = df_filtered_columns[df_filtered_columns['Monthly Reach Grouping'].isna()]

In [10]:
df_nan.head()

,Market,Brand,Category,Average Monthly Reach Target,Average Monthly Reach Projected,Monthly Reach Grouping
9,United States,Benadryl,Allergy Care,0.7,NaN,NaN
19,United States,Neutrogena,Makeup,0.7,NaN,NaN
29,UK,Aveeno Baby,Baby,0.7,NaN,NaN
39,UK,OGX,Hair Care,0.7,NaN,NaN
49,Sweden,Natusan,Baby,0.7,NaN,NaN


In [12]:
# Output each dataframe into a sheet in a new excel file called "Data_output.xlsx"
with pd.ExcelWriter('Data_output.xlsx') as writer:
    df_filtered_columns.to_excel(writer, sheet_name='All Market and Brands', index=False)
    df_under_invested.to_excel(writer, sheet_name='Under Invested', index=False)
    df_correctly_invested.to_excel(writer, sheet_name='Correctly Invested', index=False)
    df_over_invested.to_excel(writer, sheet_name='Over Invested', index=False)
    df_nan.to_excel(writer, sheet_name='No Data', index=False)

In [ ]:
# Load the workbook and expand all column widths to fit the data
book = load_workbook('./Data_output.xlsx')

for sheet in book.worksheets:
    for column in sheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the workbook
book.save('Data_output.xlsx')